<a href="https://colab.research.google.com/github/guysa-AMG/meta-rag/blob/main/Metallm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>South African carbon Emission

In [3]:
!pip install -r requirements.txt

In [2]:
import os
from getpass import getpass
import nest_asyncio
from llama_index.core import Settings
from llama_index.llms.openrouter import OpenRouter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.llama_pack import download_llama_pack
from google.colab import userdata
import requests
import re

nest_asyncio.apply()

In [4]:
os.environ["OPEN_ROUTER_API"] = userdata.get("OPEN_ROUTER_API")
if os.environ["OPEN_ROUTER_API"] is None:
  os.environ["OPEN_ROUTER_API"] = getpass("Enter your OpenRouter API key: ")


In [5]:
llm = OpenRouter(
    api_key=os.environ["OPEN_ROUTER_API"],
    model="meta-llama/llama-3.3-70b-instruct:free",
    max_tokens=512,
    temperature=0.1,
    timeout=60,
    system_prompt=(
        "You are an expert RAG system that answers ONLY using the provided context. "
        "Never hallucinate. Never guess. If the answer is not in the context, say so. "
        "Provide short, clear, factual responses with 2–4 evidence bullets."
    ),
)
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = llm
Settings.embed_model = embed_model


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6943226c-9004-4925-b626-433cee6f180f)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/vocab.txt
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2f66c95f-cab7-47bb-be9b-e53341443643)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/vocab.txt
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d2bc6ff8-9834-434b-8d9c-087b1127cc86)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/vocab.txt
Retrying in 4s [Retry 3/5].


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from utils import download_pdf_from_drive


In [7]:
file_id ="1gxGBfglAi93cnw5IAtPPNB4p8oSZqMOd"

DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)
pdf_path = os.path.join(DATA_DIR, "source.pdf")

download_pdf_from_drive(file_id,pdf_path)


[!] PDF downloaded → data/source.pdf


In [8]:
documents = SimpleDirectoryReader(input_files=[pdf_path]).load_data()
print(f"[!] Loaded {len(documents)} Pages.")

Loaded 494 Pages.


In [9]:
semantic_embed = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
parser = SemanticSplitterNodeParser( buffer_size=3,breakpoint_percentile_threshold=95, embed_model=semantic_embed)
nodes = parser.get_nodes_from_documents(documents)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f44a0f5e-ddeb-46e4-bfd9-0214a7521f4a)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/./README.md
Retrying in 1s [Retry 1/5].


In [10]:
for n in nodes:
    n.metadata["source"]     = pdf_path
    n.metadata["chunk_type"] = "semantic"

print(f"[!] Created {len(nodes)} semantic nodes.")

 Created 844 semantic nodes.


In [11]:
from llama_index.core.llama_pack import download_llama_pack

QueryRewritingRetrieverPack = download_llama_pack(
    "QueryRewritingRetrieverPack",
    "./query_rewriting_pack",
)

query_rewriting_pack = QueryRewritingRetrieverPack(
    nodes,
    chunk_size=256,
    vector_similarity_top_k=8,
    fusion_similarity_top_k=8,
    num_queries=6,
)

print("AQF RAG Engine Ready!")

AQF RAG Engine Ready!


In [12]:
def safe_rag_run(question, retries=3):
    for attempt in range(retries):
        try:
            resp = query_rewriting_pack.run(question)

            if resp is None or str(resp).strip() == "":
                raise ValueError("Empty LLM response.")
            return resp

        except Exception as e:
            print(f"Error: {e}")
            print(f"Retrying ({attempt+1}/{retries})...")

    return "[x] Could not generate a valid answer after retries."

In [13]:
while True:
    user_question = input("What would you like to know?: ").strip()
    if user_question.lower() == "q":
        break
    response = safe_rag_run(user_question)
    print("\n──────────────────────────────────────────────")
    print("? QUESTION:")
    print(user_question)
    print("\n# ANSWER:")
    print(response)
    print("──────────────────────────────────────────────\n")

What would you like to know?: what is is this about
Generated queries:
What is the main topic of this text
Explain the context of this information
Can you summarize what this is about
What is the central idea of this subject
Provide an overview of the main theme discussed here

──────────────────────────────────────────────
❓ QUESTION:
what is is this about

🧠 ANSWER:
This appears to be about:
* Emissions and fuel consumption data
* Land use and conversion
* Greenhouse gas inventory and reporting
* Data collection and calculation methods for emissions estimates 

Evidence:
* Table 3.18 shows fuel consumption data in the manufacturing industries and construction category
* Page 491 discusses land conversion and emissions
* Page 65 mentions emissions calculations and data collection methods
* Page 151 mentions the manufacturing industries and construction category
──────────────────────────────────────────────

What would you like to know?: are emission increasing
Generated queries:
Are 

KeyboardInterrupt: Interrupted by user